In [ ]:
import boto3
events = boto3.client("events", region_name=region)

# Define a daily schedule rule (EventBridge cron expression for once a day at midnight UTC)
rule_name = "AutopilotRetrainingDaily"
schedule_expression = "cron(0 0 * * ? *)"  # Every day at 00:00 UTC
events.put_rule(
    Name=rule_name,
    ScheduleExpression=schedule_expression,
    State="ENABLED",
    Description="Daily retraining trigger for SageMaker Autopilot"
)

# Target: SageMaker Pipeline (replace <PipelineName> with an actual pipeline name if you have one)
pipeline_name = "<YourRetrainingPipelineName>"
pipeline_arn = f"arn:aws:sagemaker:{region}:{boto3.client('sts').get_caller_identity()['Account']}:pipeline/{pipeline_name}"
# When targeting a SageMaker Pipeline, EventBridge needs an IAM role to invoke StartPipelineExecution
pipeline_exec_role = "<EventBridge_Invoke_Role_ARN>"  # Role with SageMaker StartPipelineExecution permission
events.put_targets(
    Rule=rule_name,
    Targets=[{
        "Id": "SageMakerPipelineTarget",
        "Arn": pipeline_arn,
        "RoleArn": pipeline_exec_role,
        "SageMakerPipelineParameters": {
            "PipelineParameterList": [
                # (Optional) specify pipeline parameters here if needed, e.g. new S3 data paths
            ]
        }
    }]
)
print(f"Scheduled EventBridge rule {rule_name} to trigger pipeline {pipeline_name} daily.")
